In [3]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = 'sk-proj-Mg8KiNNKLAlC9bNcqk1oT3BlbkFJ83Ha86GAIh7vVnvrhVPU'

In [11]:
def handle_status_intent(query, df):
    # Extract customer_id and order_id from the query
    customer_id = 1  # Example customer ID
    order_id = 101  # Example order ID

    response = get_cash_status(customer_id, order_id, df)
    return response

def handle_discoverability_intent(query):
    # Further classify the discoverability intent
    if "earned Cash" in query:
        return "This is a response for Difficulty discovering earned Cash."
    elif "items with Cash" in query or "offers with Cash" in query:
        return "This is a response for Difficulty discovering items/offers with Cash."
    else:
        return "Discoverability intent not recognized."

def handle_usability_intent(query):
    return "This is a response for Usability Intent."

def handle_faq_intent(query):
    return "This is a response for FAQ Intent."

In [12]:

def classify_intent(query):
    prompt = (
        "Classify the intent of the following query: '{query}' into one of the following intents: "
        "'Status Intent', 'Discoverability Intent', 'Usability Intent', 'FAQ Intent'. "
        "Here are some examples for each intent:\n\n"
        "Status Intent:\n"
        "1. What’s the status of my Cash?\n"
        "2. Have I earned any Walmart Cash?\n"
        "3. Can you tell me if my Cash is pending?\n\n"
        "Discoverability Intent:\n"
        "1. Where can I find my Cash?\n"
        "2. How do I see the Cash I've earned?\n"
        "3. Show me my earned Cash.\n"
        "4. Which items have Cash?\n"
        "5. What offers can get me Cash?\n"
        "6. Can you list items with Cash offers?\n\n"
        "Usability Intent:\n"
        "1. How can I use/redeem my Cash?\n"
        "2. What can I use my Walmart Cash for?\n"
        "3. Tell me how to redeem my Cash.\n\n"
        "FAQ Intent:\n"
        "1. What is Cash?\n"
        "2. How can I earn Cash?\n"
        "3. Explain Walmart Cash to me.\n\n"
        "Classify the query accurately based on these examples."
    ).format(query=query)

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100
    )
    intent = response.choices[0].text.strip()
    return intent

In [28]:
def chatbot(query, df):
    intent = classify_intent(query)

    if intent == "Status Intent":
        response = handle_status_intent(query, df)
    elif intent == "Discoverability Intent":
        response = handle_discoverability_intent(query)
    elif intent == "Usability Intent":
        response = handle_usability_intent(query)
    elif intent == "FAQ Intent":
        response = handle_faq_intent(query)
    else:
        response = "I'm sorry, I didn't understand your query."

    return response

In [27]:
def get_cash_status(customer_id, order_id, df):
    # Find the specific order for the customer
    order = df[(df['CustomerId'] == customer_id) & (df['OrderId'] == order_id)]
    if order.empty:
        return "Order not found."

    order = order.iloc[0]

    # Check the eligibility and status conditions
    if not order['EligibleForCash']:
        return "The items in this order are not eligible for Walmart Cash."

    if not order['ClipMade']:
        return "You did not clip the eligible items for Walmart Cash."

    if not order['MustBuyCriteriaMet']:
        return "The Must Buy criteria were not met for this order."

    if not order['PurchaseWithinRedeemWindow']:
        return "The purchase was not made within the redeem window."

    if order['OrderStatus'] in ['Canceled', 'Returned', 'Substituted', 'Unavailable']:
        return f"The order was {order['OrderStatus']}, so you are not eligible for Walmart Cash."

    if order['OrderStatus'] == 'Pending':
        return "The order is still pending. Your Walmart Cash will be processed once the order is delivered."

    if order['CashEarned'] > 0:
        return f"You have earned ${order['CashEarned']} Walmart Cash on {order['CashEarnedDate']}."

    return "There was an issue with earning Walmart Cash. Please contact customer support if it has been more than 3 days since delivery."



In [19]:


# Create a dummy dataset
data = {
    'CustomerId': [1, 2, 3, 4],
    'OrderId': [101, 102, 103, 104],
    'OrderDate': ['2024-06-01', '2024-06-02', '2024-06-03', '2024-06-04'],
    'OrderAmount': [100, 150, 200, 250],
    'ItemsPurchased': [['item1', 'item2'], ['item3'], ['item4', 'item5'], ['item6']],
    'EligibleForCash': [True, True, False, True],
    'ClipMade': [True, True, False, True],
    'MustBuyCriteriaMet': [True, True, False, False],
    'PurchaseWithinRedeemWindow': [True, True, False, True],
    'OrderStatus': ['Delivered', 'Pending', 'Delivered', 'Returned'],
    'CashEarned': [10, 0, 0, 0],
    'CashEarnedDate': ['2024-06-04', None, None, None]
}

df = pd.DataFrame(data)

In [20]:
df

,CustomerId,OrderId,OrderDate,OrderAmount,ItemsPurchased,EligibleForCash,ClipMade,MustBuyCriteriaMet,PurchaseWithinRedeemWindow,OrderStatus,CashEarned,CashEarnedDate
0,1,101,2024-06-01,100,"[item1, item2]",True,True,True,True,Delivered,10,2024-06-04
1,2,102,2024-06-02,150,[item3],True,True,True,True,Pending,0,None
2,3,103,2024-06-03,200,"[item4, item5]",False,False,False,False,Delivered,0,None
3,4,104,2024-06-04,250,[item6],True,True,False,True,Returned,0,None


In [29]:
# Example query to the chatbot
query = "Toys eligibl for Cash"
response = chatbot(query, df)
print(response)

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
